In [261]:
import sys
print(sys.executable)

C:\Users\Johanna\anaconda3\envs\AI_project\python.exe


In [267]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.layers import  Dropout
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from tensorflow.keras.layers import Input
import joblib
import matplotlib.pyplot as plt
from datetime import time


In [134]:
#df_lagged_all = pd.read_parquet("data/processed/df_lagged_30_all.parquet")
df_lagged_all = pd.read_parquet("data/processed/df_lagged_all.parquet")
df_2_full = pd.read_parquet("data/processed/df_2_full_v2.parquet")

df_all=pd.read_parquet("data/processed/df_all.parquet")
df_all_peak= df_all.set_index("Datetime").between_time("07:15", "08:30").reset_index()



Variables

In [5]:
target_sensor = 1076
same_portal="55620"
neighbour_portal = "56160"

same_portal_sensors = df_2_full[df_2_full['PORTAL_clean'] == same_portal]['DP_ID'].unique()
same_sensors = [s for s in same_portal_sensors if s != target_sensor]


neighbour_sensors = df_2_full[df_2_full['PORTAL_clean'] == neighbour_portal]['DP_ID'].unique()
all_sensors=df_2_full['DP_ID'].unique()
except_target_sensors = [s for s in all_sensors if s != target_sensor]




In [69]:
nlags=15
#nlags=30

In [263]:
flow_features_same_nl = [
    f'SENSOR_{sensor}_FLOW'
    for sensor in same_sensors

]
flow_features_neighbour_nl = [
    f'SENSOR_{sensor}_FLOW'
    for sensor in neighbour_sensors

]
speed_features_same_nl = [
    f'SENSOR_{sensor}_SPEED'
    for sensor in same_sensors
]
speed_features_neighbour_nl = [
    f'SENSOR_{sensor}_SPEED'
    for sensor in neighbour_sensors
]

In [264]:
flow_features_same = [
    f'SENSOR_{sensor}_FLOW_lag_{i+1}'
    for sensor in same_sensors
    for i in range(nlags)
]
flow_features_neighbour = [
    f'SENSOR_{sensor}_FLOW_lag_{i+1}'
    for sensor in neighbour_sensors
    for i in range(nlags)
]
speed_features_same = [
    f'SENSOR_{sensor}_SPEED_lag_{i+1}'
    for sensor in same_sensors
    for i in range(nlags)
]
speed_features_neighbour = [
    f'SENSOR_{sensor}_SPEED_lag_{i+1}'
    for sensor in neighbour_sensors
    for i in range(nlags)
]        

Train -Test Split

In [269]:
df_train, df_test = train_test_split(df_lagged_all, test_size=0.2, random_state=42)



Scaling

In [276]:
# FLOW same
scaler_flow_same = StandardScaler()
X_train_scaled_flow_same = scaler_flow_same.fit_transform(df_train[flow_features_same])
X_test_scaled_flow_same  = scaler_flow_same.transform(df_test[flow_features_same])
joblib.dump(scaler_flow_same, "model/scaler_flow_same.pkl")

# SPEED same
scaler_speed_same = StandardScaler()
X_train_scaled_speed_same = scaler_speed_same.fit_transform(df_train[speed_features_same])
X_test_scaled_speed_same  = scaler_speed_same.transform(df_test[speed_features_same])
joblib.dump(scaler_flow_same, "model/scaler_speed_same.pkl")


# FLOW neighbour
scaler_flow_neigh = StandardScaler()
X_train_scaled_flow_neighbour = scaler_flow_neigh.fit_transform(df_train[flow_features_neighbour])
X_test_scaled_flow_neighbour  = scaler_flow_neigh.transform(df_test[flow_features_neighbour])
joblib.dump(scaler_flow_same, "model/scaler_flow_neighbour.pkl")


# SPEED neighbour
scaler_speed_neighbour = StandardScaler()
X_train_scaled_speed_neighbour = scaler_speed_neighbour.fit_transform(df_train[speed_features_neighbour])
X_test_scaled_speed_neighbour  = scaler_speed_neighbour.transform(df_test[speed_features_neighbour])
joblib.dump(scaler_flow_same, "model/scaler_speed_neighbour.pkl")



['model/scaler_speed_neighbour.pkl']

Linear Regression

In [270]:
#same portal
model_flow = LinearRegression()
model_flow.fit(df_train[flow_features_same], df_train['FLOW_future_sum'])

model_speed = LinearRegression()
model_speed.fit(df_train[speed_features_same], df_train['SPEED_future_mean'])


# Vorhersage
y_pred = model_flow.predict(df_test[flow_features_same])
y_test=df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")


y_pred = model_speed.predict(df_test[speed_features_same])
y_test=df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

Baseline Linear Regression (FLOW) -> RMSE: 33.543, MAE: 23.809, R2: 0.836
Baseline Linear Regression (SPEED) -> RMSE: 0.861, MAE: 0.462, R2: 0.709


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [271]:
#neighbour portal
model_flow = LinearRegression()
model_flow.fit(df_train[flow_features_neighbour], df_train['FLOW_future_sum'])

model_speed = LinearRegression()
model_speed.fit(df_train[speed_features_neighbour], df_train['SPEED_future_mean'])


#prediction
y_pred = model_flow.predict(df_test[flow_features_neighbour])
y_test=df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")


y_pred = model_speed.predict(df_test[speed_features_neighbour])
y_test=df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

Baseline Linear Regression (FLOW) -> RMSE: 28.474, MAE: 19.559, R2: 0.882
Baseline Linear Regression (SPEED) -> RMSE: 1.051, MAE: 0.513, R2: 0.566


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


XGBoost

In [272]:

model_flow = XGBRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_flow.fit(df_train[flow_features_same], df_train['FLOW_future_sum'])

model_speed = XGBRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_speed.fit(df_train[speed_features_same], df_train['SPEED_future_mean'])




y_pred = model_flow.predict(df_test[flow_features_same])
y_test = df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

y_pred = model_speed.predict(df_test[speed_features_same])
y_test = df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

XGBoost Regression (FLOW) -> RMSE: 28.647, MAE: 20.634, R²: 0.880
XGBoost Regression (SPEED) -> RMSE: 0.810, MAE: 0.401, R²: 0.742


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [273]:
model_flow = XGBRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_flow.fit(df_train[flow_features_neighbour], df_train['FLOW_future_sum'])

model_speed = XGBRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_speed.fit(df_train[speed_features_neighbour], df_train['SPEED_future_mean'])




y_pred = model_flow.predict(df_test[flow_features_neighbour])
y_test = df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

y_pred = model_speed.predict(df_test[speed_features_neighbour])
y_test = df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

XGBoost Regression (FLOW) -> RMSE: 24.460, MAE: 17.192, R²: 0.913
XGBoost Regression (SPEED) -> RMSE: 0.936, MAE: 0.424, R²: 0.655


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


xgboost with hyperparameter tuning

In [220]:
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.001,0.005,0.01, 0.05],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0]
}

In [251]:
results_df = pd.DataFrame(columns=["Type", "MAE", "RMSE", "R²"])

In [253]:
def add_result(results_df, type, y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2   = r2_score(y_true, y_pred)

    new_row = {
        "Type": type,
        "MAE": mae,
        "RMSE": rmse,
        "R²": r2
    }

    return pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)

In [257]:
#same portal
random_search_flow = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30, 
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)

random_search_flow.fit(df_train[flow_features_same], df_train["FLOW_future_sum"])
print("Beste Parameter (FLOW):", random_search_flow.best_params_)
model_flow = random_search_flow.best_estimator_
model_flow.save_model("model/xgb_flow_same.json")

features_used = df_train[flow_features_same].columns.tolist()
joblib.dump(features_used, "model/features_flow_same.pkl")

y_pred = model_flow.predict(df_test[flow_features_same])
y_test = df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

results_df = add_result(results_df, "FLOW -same portal", y_test, y_pred)


print(f"XGBoost Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


random_search_speed = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30,  
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)


random_search_speed.fit(df_train[speed_features_same], df_train["SPEED_future_mean"])
print("Beste Parameter (FLOW):", random_search_speed.best_params_)
model_speed = random_search_speed.best_estimator_
model_speed.save_model("model/xgb_speed_same.json")

features_used = df_train[speed_features_same].columns.tolist()
joblib.dump(features_used, "model/features_speed_same.pkl")


y_pred = model_speed.predict(df_test[speed_features_same])
y_test = df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

results_df = add_result(results_df, "SPEED -same portal", y_test, y_pred)


print(f"XGBoost Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Beste Parameter (FLOW): {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
XGBoost Regression (FLOW) -> RMSE: 28.311, MAE: 20.292, R²: 0.883
Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\AppData\Local\Temp\ipykernel_10492\1773143569.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=Tru

Beste Parameter (FLOW): {'subsample': 0.6, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
XGBoost Regression (SPEED) -> RMSE: 0.812, MAE: 0.401, R²: 0.740


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [258]:
#neighbour portal
random_search_flow = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30,  
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)

random_search_flow.fit(df_train[flow_features_neighbour], df_train["FLOW_future_sum"])
print("Beste Parameter (FLOW):", random_search_flow.best_params_)
model_flow = random_search_flow.best_estimator_
model_flow.save_model("model/xgb_flow_neighbour.json")

features_used = df_train[flow_features_neighbour].columns.tolist()
joblib.dump(features_used, "model/features_flow_neighbour.pkl")


y_pred = model_flow.predict(df_test[flow_features_neighbour])
y_test = df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

results_df = add_result(results_df, "FLOW -neighbour portal", y_test, y_pred)


print(f"XGBoost Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


random_search_speed = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30, 
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)


random_search_speed.fit(df_train[speed_features_neighbour], df_train["SPEED_future_mean"])
print("Beste Parameter (FLOW):", random_search_speed.best_params_)
model_speed = random_search_speed.best_estimator_
model_speed.save_model("model/xgb_speed_neighbour.json")

features_used = df_train[speed_features_neighbour].columns.tolist()
joblib.dump(features_used, "model/features_speed_neighbour.pkl")


y_pred = model_speed.predict(df_test[speed_features_neighbour])
y_test = df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

results_df = add_result(results_df, "SPEED -neighbour portal", y_test, y_pred)


print(f"XGBoost Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Beste Parameter (FLOW): {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
XGBoost Regression (FLOW) -> RMSE: 24.096, MAE: 16.871, R²: 0.915
Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Beste Parameter (FLOW): {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
XGBoost Regression (SPEED) -> RMSE: 0.930, MAE: 0.420, R²: 0.660


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [260]:
results_df.sort_values("Type")

,Type,MAE,RMSE,R²
2,FLOW -neighbour portal,16.871492,24.095540,0.915146
0,FLOW -same portal,20.291550,28.310582,0.882863
3,SPEED -neighbour portal,0.419800,0.930045,0.659704
1,SPEED -same portal,0.401327,0.812237,0.740454


Feedforward Neural Network (NN)

In [278]:
#same portal 

model_flow= Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled_flow_same.shape[1],)),  
    Dense(32, activation='relu'),                                           
    Dense(16, activation='relu'),                                         
    Dense(1)                                                              
])

model_flow.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_flow = model_flow.fit(
    X_train_scaled_flow_same, df_train["FLOW_future_sum"],
    validation_split=0.2,  
    epochs=50,
    batch_size=32,
    verbose=0
)



y_pred = model_flow.predict(X_test_scaled_flow_same)
y_test=df_test["FLOW_future_sum"]

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


model_speed = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled_speed_same.shape[1],)),  
    Dense(32, activation='relu'),                                          
    Dense(16, activation='relu'),                                           
    Dense(1)                                                                
])

model_speed.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_speed = model_speed.fit(
    X_train_scaled_speed_same, df_train["SPEED_future_mean"],
    validation_split=0.2,  
    epochs=50,
    batch_size=32,
    verbose=0
)


y_pred = model_speed.predict(X_test_scaled_speed_same)
y_test=df_test["SPEED_future_mean"]
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")



457/457 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fast forward NN (FLOW) -> RMSE: 29.248, MAE: 20.778, R²: 0.875
457/457 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Fast forward NN (SPEED) -> RMSE: 0.819, MAE: 0.433, R²: 0.736


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [279]:
#neighbour portal
model_flow= Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled_flow_neighbour.shape[1],)),  
    Dense(32, activation='relu'),                                           
    Dense(16, activation='relu'),                                           
    Dense(1)                                                                 
])

model_flow.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_flow = model_flow.fit(
    X_train_scaled_flow_neighbour, df_train["FLOW_future_sum"],
    validation_split=0.2,  
    epochs=50,
    batch_size=32,
    verbose=0
)



y_pred = model_flow.predict(X_test_scaled_flow_neighbour)
y_test=df_test["FLOW_future_sum"]

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")



model_speed = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled_speed_neighbour.shape[1],)), 
    Dense(32, activation='relu'),                          
    Dense(16, activation='relu'),         
    Dense(1)                                                   
])

model_speed.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_speed = model_speed.fit(
    X_train_scaled_speed_neighbour, df_train["SPEED_future_mean"],
    validation_split=0.2,  
    epochs=50,
    batch_size=32,
    verbose=0
)


y_pred = model_speed.predict(X_test_scaled_speed_neighbour)
y_test=df_test["SPEED_future_mean"]
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")



C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


457/457 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Fast forward NN (FLOW) -> RMSE: 24.794, MAE: 17.303, R²: 0.910


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


457/457 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Fast forward NN (SPEED) -> RMSE: 0.969, MAE: 0.458, R²: 0.631


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


NN with GridSearch

In [87]:
#same portal-flow

y_train = df_train["FLOW_future_sum"]
y_test = df_test["FLOW_future_sum"]

number_neurons = [16, 32, 64, 128]
number_layers = [2, 3,4]
dropout_rates = [0.0, 0.05, 0.1]

grid_search_df = pd.DataFrame(columns=["number_neurons","number_layers","dropout_rate","train_rmse", "val_rmse"])

for neurons in number_neurons:
    for num_layer in number_layers:
        for dropout_rate in dropout_rates:
            model = Sequential()
            for _ in range(num_layer):
                model.add(Dense(neurons, activation='relu'))
                if dropout_rate > 0:
                    model.add(Dropout(dropout_rate))
            model.add(Dense(1))
            optimizer = Adam(learning_rate=0.01)
            model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    
            early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
    
            hist = model.fit(
                X_train_scaled_flow_same, y_train,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr],
                verbose=0
            )
    
            min_err = min(hist.history['rmse'])
            min_val_err = min(hist.history['val_rmse'])
            this_df = pd.DataFrame({
                "number_neurons": [neurons],
                "number_layers": [num_layer],
                "dropout_rate":[dropout_rate],
                "train_rmse": [min_err],
                "val_rmse": [min_val_err]
            })
            grid_search_df = pd.concat([grid_search_df, this_df], axis=0)
            print(neurons, num_layer,dropout_rate, "done")


grid_search_df.reset_index(drop=True, inplace=True)
print(grid_search_df)
best_config = grid_search_df.loc[grid_search_df['val_rmse'].idxmin()]
best_neurons = int(best_config['number_neurons'])
best_layers = int(best_config['number_layers'])
best_dropout = int(best_config['dropout_rate'])

# Modell with best parameters
best_model = Sequential()
for _ in range(best_layers):
    best_model.add(Dense(best_neurons, activation='relu'))
    if best_dropout > 0:
        best_model.add(Dropout(best_dropout))
best_model.add(Dense(1))

optimizer = Adam(learning_rate=0.01)
best_model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
checkpoint = ModelCheckpoint("model/NN_model_flow_same_best.keras", monitor='val_rmse', save_best_only=True, mode='min')


best_model.fit(
    X_train_scaled_flow_same, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=0
)


y_pred = best_model.predict(X_test_scaled_flow_same)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Best FLOW NN -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")






C:\Users\Johanna\AppData\Local\Temp\ipykernel_10492\308595228.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grid_search_df = pd.concat([grid_search_df, it_df], axis=0)


16 2 0.0 done
16 2 0.05 done
16 2 0.1 done
16 3 0.0 done
16 3 0.05 done
16 3 0.1 done
16 4 0.0 done
16 4 0.05 done
16 4 0.1 done
32 2 0.0 done
32 2 0.05 done
32 2 0.1 done
32 3 0.0 done
32 3 0.05 done
32 3 0.1 done
32 4 0.0 done
32 4 0.05 done
32 4 0.1 done
64 2 0.0 done
64 2 0.05 done
64 2 0.1 done
64 3 0.0 done
64 3 0.05 done
64 3 0.1 done
64 4 0.0 done
64 4 0.05 done
64 4 0.1 done
128 2 0.0 done
128 2 0.05 done
128 2 0.1 done
128 3 0.0 done
128 3 0.05 done
128 3 0.1 done
128 4 0.0 done
128 4 0.05 done
128 4 0.1 done
   number_neurons number_layers  dropout_rate  train_rmse   val_rmse
0              16             2          0.00   29.371403  29.733335
1              16             2          0.05   37.075043  33.153324
2              16             2          0.10   42.041279  33.635246
3              16             3          0.00   27.912714  28.976200
4              16             3          0.05   39.897415  33.503639
5              16             3          0.10   43.846443  34

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [89]:
#same portal-speed
y_train = df_train["SPEED_future_mean"]
y_test = df_test["SPEED_future_mean"]

number_neurons = [16, 32, 64, 128]
number_layers = [2, 3,4]
dropout_rates = [0.0, 0.05, 0.1]

grid_search_df = pd.DataFrame(columns=["number_neurons","number_layers","dropout_rate","train_rmse", "val_rmse"])

for neurons in number_neurons:
    for num_layer in number_layers:
        for dropout_rate in dropout_rates:
            model = Sequential()
            for _ in range(num_layer):
                model.add(Dense(neurons, activation='relu'))
                if dropout_rate > 0:
                    model.add(Dropout(dropout_rate))
            model.add(Dense(1))
            optimizer = Adam(learning_rate=0.01)
            model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    
            early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
    
            hist = model.fit(
                X_train_scaled_speed_same, y_train,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr],
                verbose=0
            )
    
            min_err = min(hist.history['rmse'])
            min_val_err = min(hist.history['val_rmse'])
            this_df = pd.DataFrame({
                "number_neurons": [neurons],
                "number_layers": [num_layer],
                "dropout_rate":[dropout_rate],
                "train_rmse": [min_err],
                "val_rmse": [min_val_err]
            })
            grid_search_df = pd.concat([grid_search_df, this_df], axis=0)
            print(neurons, num_layer,dropout_rate, "done")


grid_search_df.reset_index(drop=True, inplace=True)
print(grid_search_df)
best_config = grid_search_df.loc[grid_search_df['val_rmse'].idxmin()]
best_neurons = int(best_config['number_neurons'])
best_layers = int(best_config['number_layers'])
best_dropout = int(best_config['dropout_rate'])

#model with best parameters
best_model = Sequential()
for _ in range(best_layers):
    best_model.add(Dense(best_neurons, activation='relu'))
    if best_dropout > 0:
        best_model.add(Dropout(best_dropout))
best_model.add(Dense(1))

optimizer = Adam(learning_rate=0.01)
best_model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
checkpoint = ModelCheckpoint("model/NN_model_speed_same_best.keras", monitor='val_rmse', save_best_only=True, mode='min')


best_model.fit(
    X_train_scaled_speed_same, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=0
)

# test
y_pred = best_model.predict(X_test_scaled_speed_same)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Best SPEED NN -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")






C:\Users\Johanna\AppData\Local\Temp\ipykernel_10492\867997906.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grid_search_df = pd.concat([grid_search_df, it_df], axis=0)


16 2 0.0 done
16 2 0.05 done
16 2 0.1 done
16 3 0.0 done
16 3 0.05 done
16 3 0.1 done
16 4 0.0 done
16 4 0.05 done
16 4 0.1 done
32 2 0.0 done
32 2 0.05 done
32 2 0.1 done
32 3 0.0 done
32 3 0.05 done
32 3 0.1 done
32 4 0.0 done
32 4 0.05 done
32 4 0.1 done
64 2 0.0 done
64 2 0.05 done
64 2 0.1 done
64 3 0.0 done
64 3 0.05 done
64 3 0.1 done
64 4 0.0 done
64 4 0.05 done
64 4 0.1 done
128 2 0.0 done
128 2 0.05 done
128 2 0.1 done
128 3 0.0 done
128 3 0.05 done
128 3 0.1 done
128 4 0.0 done
128 4 0.05 done
128 4 0.1 done
   number_neurons number_layers  dropout_rate  train_rmse  val_rmse
0              16             2          0.00    0.805148  0.790606
1              16             2          0.05    0.973746  0.811842
2              16             2          0.10    0.822249  0.793180
3              16             3          0.00    0.733872  0.737610
4              16             3          0.05    0.812134  0.754808
5              16             3          0.10    0.878491  0.784833

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [91]:
#neighbour portal-flow
y_train = df_train["FLOW_future_sum"]
y_test = df_test["FLOW_future_sum"]

number_neurons = [16, 32, 64, 128]
number_layers = [2, 3,4]
dropout_rates = [0.0, 0.05, 0.1]

grid_search_df = pd.DataFrame(columns=["number_neurons","number_layers","dropout_rate","train_rmse", "val_rmse"])

for neurons in number_neurons:
    for num_layer in number_layers:
        for dropout_rate in dropout_rates:
            model = Sequential()
            for _ in range(num_layer):
                model.add(Dense(neurons, activation='relu'))
                if dropout_rate > 0:
                    model.add(Dropout(dropout_rate))
            model.add(Dense(1))
            optimizer = Adam(learning_rate=0.01)
            model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    
            early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)

            hist = model.fit(
                X_train_scaled_flow_neighbour, y_train,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr],
                verbose=0
            )
    
            min_err = min(hist.history['rmse'])
            min_val_err = min(hist.history['val_rmse'])
            this_df = pd.DataFrame({
                "number_neurons": [neurons],
                "number_layers": [num_layer],
                "dropout_rate":[dropout_rate],
                "train_rmse": [min_err],
                "val_rmse": [min_val_err]
            })
            grid_search_df = pd.concat([grid_search_df, this_df], axis=0)
            print(neurons, num_layer,dropout_rate, "done")


grid_search_df.reset_index(drop=True, inplace=True)
print(grid_search_df)
best_config = grid_search_df.loc[grid_search_df['val_rmse'].idxmin()]
best_neurons = int(best_config['number_neurons'])
best_layers = int(best_config['number_layers'])
best_dropout = int(best_config['dropout_rate'])
print(best_neurons, best_layers, best_dropout)

# model with best parameters
best_model = Sequential()
for _ in range(best_layers):
    best_model.add(Dense(best_neurons, activation='relu'))
    if best_dropout > 0:
        best_model.add(Dropout(best_dropout))
best_model.add(Dense(1))

optimizer = Adam(learning_rate=0.01)
best_model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
checkpoint = ModelCheckpoint("model/NN_model_flow_neighbour_best.keras", monitor='val_rmse', save_best_only=True, mode='min')


best_model.fit(
    X_train_scaled_flow_neighbour, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=0
)


y_pred = best_model.predict(X_test_scaled_flow_neighbour)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Best FLOW NN -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")






C:\Users\Johanna\AppData\Local\Temp\ipykernel_10492\2257366272.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grid_search_df = pd.concat([grid_search_df, it_df], axis=0)


16 2 0.0 done
16 2 0.05 done
16 2 0.1 done
16 3 0.0 done
16 3 0.05 done
16 3 0.1 done
16 4 0.0 done
16 4 0.05 done
16 4 0.1 done
32 2 0.0 done
32 2 0.05 done
32 2 0.1 done
32 3 0.0 done
32 3 0.05 done
32 3 0.1 done
32 4 0.0 done
32 4 0.05 done
32 4 0.1 done
64 2 0.0 done
64 2 0.05 done
64 2 0.1 done
64 3 0.0 done
64 3 0.05 done
64 3 0.1 done
64 4 0.0 done
64 4 0.05 done
64 4 0.1 done
128 2 0.0 done
128 2 0.05 done
128 2 0.1 done
128 3 0.0 done
128 3 0.05 done
128 3 0.1 done
128 4 0.0 done
128 4 0.05 done
128 4 0.1 done
   number_neurons number_layers  dropout_rate  train_rmse   val_rmse
0              16             2          0.00   24.768108  25.254890
1              16             2          0.05   29.870770  26.075983
2              16             2          0.10   33.876259  25.873125
3              16             3          0.00   23.947803  24.581074
4              16             3          0.05   32.065853  28.567739
5              16             3          0.10   34.205040  26

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [93]:
#neighbour portal-speed

y_train = df_train["SPEED_future_mean"]
y_test = df_test["SPEED_future_mean"]

number_neurons = [16, 32, 64, 128]
number_layers = [2, 3,4]
dropout_rates = [0.0, 0.05, 0.1]

grid_search_df = pd.DataFrame(columns=["number_neurons","number_layers","dropout_rate","train_rmse", "val_rmse"])

for neurons in number_neurons:
    for num_layer in number_layers:
        for dropout_rate in dropout_rates:
            model = Sequential()
            for _ in range(num_layer):
                model.add(Dense(neurons, activation='relu'))
                if dropout_rate > 0:
                    model.add(Dropout(dropout_rate))
            model.add(Dense(1))
            optimizer = Adam(learning_rate=0.01)
            model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    
            early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
    
            # Optional: speichern der besten Gewichte
            #checkpoint = ModelCheckpoint("model/NN_model_speed_neighbour_best.keras", monitor='val_rmse', save_best_only=True, mode='min')
    
            hist = model.fit(
                X_train_scaled_speed_neighbour, y_train,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr],
                verbose=0
            )
    
            # Ergebnisse speichern
            min_err = min(hist.history['rmse'])
            min_val_err = min(hist.history['val_rmse'])
            it_df = pd.DataFrame({
                "number_neurons": [neurons],
                "number_layers": [num_layer],
                "dropout_rate":[dropout_rate],
                "train_rmse": [min_err],
                "val_rmse": [min_val_err]
            })
            grid_search_df = pd.concat([grid_search_df, it_df], axis=0)
            print(neurons, num_layer,dropout_rate, "done")

# Übersicht
grid_search_df.reset_index(drop=True, inplace=True)
print(grid_search_df)
best_config = grid_search_df.loc[grid_search_df['val_rmse'].idxmin()]
best_neurons = int(best_config['number_neurons'])
best_layers = int(best_config['number_layers'])
best_dropout = int(best_config['dropout_rate'])
print(best_neurons, best_layers, best_dropout)

# Modell mit besten Parametern
best_model = Sequential()
for _ in range(best_layers):
    best_model.add(Dense(best_neurons, activation='relu'))
    if best_dropout > 0:
        best_model.add(Dropout(best_dropout))
best_model.add(Dense(1))

optimizer = Adam(learning_rate=0.01)
best_model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)
checkpoint = ModelCheckpoint("model/NN_model_speed_neighbour_best.keras", monitor='val_rmse', save_best_only=True, mode='min')

# Training
best_model.fit(
    X_train_scaled_speed_neighbour, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=0
)

# Testbewertung
y_pred = best_model.predict(X_test_scaled_speed_neighbour)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Best SPEED NN -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")






C:\Users\Johanna\AppData\Local\Temp\ipykernel_10492\326426236.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grid_search_df = pd.concat([grid_search_df, it_df], axis=0)


16 2 0.0 done
16 2 0.05 done
16 2 0.1 done
16 3 0.0 done
16 3 0.05 done
16 3 0.1 done
16 4 0.0 done
16 4 0.05 done
16 4 0.1 done
32 2 0.0 done
32 2 0.05 done
32 2 0.1 done
32 3 0.0 done
32 3 0.05 done
32 3 0.1 done
32 4 0.0 done
32 4 0.05 done
32 4 0.1 done
64 2 0.0 done
64 2 0.05 done
64 2 0.1 done
64 3 0.0 done
64 3 0.05 done
64 3 0.1 done
64 4 0.0 done
64 4 0.05 done
64 4 0.1 done
128 2 0.0 done
128 2 0.05 done
128 2 0.1 done
128 3 0.0 done
128 3 0.05 done
128 3 0.1 done
128 4 0.0 done
128 4 0.05 done
128 4 0.1 done
   number_neurons number_layers  dropout_rate  train_rmse  val_rmse
0              16             2          0.00    0.917047  0.926511
1              16             2          0.05    0.908428  0.900218
2              16             2          0.10    0.936143  0.904106
3              16             3          0.00    0.855502  0.891089
4              16             3          0.05    0.913606  0.893798
5              16             3          0.10    0.961896  0.917284

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


LSTM

In [100]:
def create_sequences(X, y, seq_length=15, horizon=15):
    Xs, ys = [], []
    for i in range(len(X) - seq_length - horizon + 1):
        Xs.append(X[i:i+seq_length])
        ys.append(y[i+seq_length:i+seq_length+horizon])
    return np.array(Xs), np.array(ys)


In [247]:

X = df_all[flow_features_neighbour_nl].values
y = df_all[f'SENSOR_{target_sensor}_FLOW'].values



X_seq, y_seq = create_sequences(X, y, seq_length=15, horizon=15)

X_scaler = StandardScaler()
y_scaler = StandardScaler()

# reshape für den Scaler (2D erwartet)
X_scaled = X_scaler.fit_transform(X_seq.reshape(-1, X_seq.shape[2])).reshape(X_seq.shape)
#y_scaled = y_scaler.fit_transform(y_seq.reshape(-1, 1))
X_seq=X_scaled
#y_seq=y_scaled


n_features = X_seq.shape[2]

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(15,n_features), return_sequences=True),
    LSTM(64, activation='tanh', return_sequences=True),
    LSTM(64, activation='tanh', return_sequences=False),
    Dense(15)  # 15 Output-Werte für 15-Minuten-Vorhersage
])

model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

model.fit(X_train, y_train, epochs=20, batch_size=32,callbacks=[early_stop, reduce_lr, checkpoint], validation_split=0.1,verbose=1)


y_pred = model.predict(X_test)





Epoch 1/20


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1642/1642 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step - loss: 28.2451 - rmse: 5.3146 - val_loss: 18.9741 - val_rmse: 4.3559 - learning_rate: 0.0010
Epoch 2/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 22s 13ms/step - loss: 16.2927 - rmse: 4.0364 - val_loss: 19.0427 - val_rmse: 4.3638 - learning_rate: 0.0010
Epoch 3/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - loss: 16.2124 - rmse: 4.0265 - val_loss: 18.6845 - val_rmse: 4.3226 - learning_rate: 0.0010
Epoch 4/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - loss: 16.0428 - rmse: 4.0053 - val_loss: 18.7960 - val_rmse: 4.3354 - learning_rate: 5.0000e-04
Epoch 5/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - loss: 15.9892 - rmse: 3.9986 - val_loss: 18.8916 - val_rmse: 4.3464 - learning_rate: 5.0000e-04
457/457 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step


In [249]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"LSTM (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

y_test_sum = y_test.sum(axis=1)
y_pred_sum = y_pred.sum(axis=1)

# Metriken berechnen
rmse = mean_squared_error(y_test_sum, y_pred_sum, squared=False)
mae = mean_absolute_error(y_test_sum, y_pred_sum)
r2 = r2_score(y_test_sum, y_pred_sum)

print(f"LSTM Sum Forecast (FLOW) → RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

print(y_pred.shape)
print(y_pred_sum.shape)

LSTM (FLOW) -> RMSE: 3.931, MAE: 2.884, R2: 0.644
LSTM Sum Forecast (FLOW) → RMSE: 35.576, MAE: 22.303, R²: 0.833
(14596, 15)
(14596,)


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [250]:

X = df_all[speed_features_neighbour_nl].values
y = df_all[f'SENSOR_{target_sensor}_SPEED'].values



X_seq, y_seq = create_sequences(X, y, seq_length=15, horizon=15)

X_scaler = StandardScaler()
y_scaler = StandardScaler()

# reshape für den Scaler (2D erwartet)
X_scaled = X_scaler.fit_transform(X_seq.reshape(-1, X_seq.shape[2])).reshape(X_seq.shape)
#y_scaled = y_scaler.fit_transform(y_seq.reshape(-1, 1))
X_seq=X_scaled
#y_seq=y_scaled


n_features = X_seq.shape[2]

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(15,n_features), return_sequences=True),
    LSTM(64, activation='tanh', return_sequences=True),
    LSTM(64, activation='tanh', return_sequences=False),
    Dense(15)  # 15 Output-Werte für 15-Minuten-Vorhersage
])

model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

model.fit(X_train, y_train, epochs=20, batch_size=32,callbacks=[early_stop, reduce_lr, checkpoint], validation_split=0.1,verbose=1)


y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"LSTM (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

y_test_sum = y_test.sum(axis=1)
y_pred_sum = y_pred.sum(axis=1)

# Metriken berechnen
rmse = mean_squared_error(y_test_sum, y_pred_sum, squared=False)
mae = mean_absolute_error(y_test_sum, y_pred_sum)
r2 = r2_score(y_test_sum, y_pred_sum)

print(f"LSTM Sum Forecast (SPEED) → RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")





Epoch 1/20


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1642/1642 ━━━━━━━━━━━━━━━━━━━━ 25s 14ms/step - loss: 21.6212 - rmse: 4.6499 - val_loss: 3.1687 - val_rmse: 1.7801 - learning_rate: 0.0010
Epoch 2/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - loss: 2.2561 - rmse: 1.5020 - val_loss: 3.0645 - val_rmse: 1.7506 - learning_rate: 0.0010
Epoch 3/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 22s 13ms/step - loss: 1.9048 - rmse: 1.3802 - val_loss: 2.6482 - val_rmse: 1.6273 - learning_rate: 0.0010
Epoch 4/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 22s 13ms/step - loss: 1.8661 - rmse: 1.3660 - val_loss: 2.8612 - val_rmse: 1.6915 - learning_rate: 5.0000e-04
Epoch 5/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 22s 13ms/step - loss: 1.8574 - rmse: 1.3629 - val_loss: 2.5974 - val_rmse: 1.6116 - learning_rate: 5.0000e-04
457/457 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step
LSTM (SPEED) -> RMSE: 2.094, MAE: 1.477, R2: -0.010
LSTM Sum Forecast (SPEED) → RMSE: 25.775, MAE: 19.042, R²: -0.015


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [158]:
df_all['FLOW_future_sum'] = (
    df_all[f'SENSOR_{target_sensor}_FLOW']
    .rolling(15, min_periods=15)
    .sum()
    .shift(-14)  # damit die Summe bei t=07:02 die Werte von 07:02–07:16 enthält
    .reset_index(level=0, drop=True)
)
df_all['SPEED_future_mean'] = (
    df_all[f'SENSOR_{target_sensor}_SPEED']
    .rolling(15, min_periods=15)
    .mean()
    .shift(-14)  # damit die Summe bei t=07:02 die Werte von 07:02–07:16 enthält
    .reset_index(level=0, drop=True)
)


print(df_all.shape)
df_all = df_all[df_all['Datetime'].dt.time <= time(9, 45)]
print(df_all.shape)

#print(df_lagged_all["Datetime"]dt.time.min(), df_lagged_all["Datetime"].dt.date.max())
#df_all.head(20)
#print(df_lagged_all.isna().sum())


(76171, 15)
(73006, 15)


In [167]:
def create_sequences_sum(X, y, seq_length=15, horizon=15):
    Xs, ys = [], []
    for i in range(len(X) - seq_length - horizon + 1):
        Xs.append(X[i:i+seq_length])
        # statt array von 15 Werten → Summe
        ys.append(y[i+seq_length:i+seq_length+horizon].sum())  
    return np.array(Xs), np.array(ys)


In [242]:
early_stop = EarlyStopping(monitor='val_rmse', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3)

            

In [236]:
split = int(len(X) * 0.8)
df_train, df_test = df_all[:split], df_all[split:]


X_train = df_train[flow_features_same].values
y_train = df_train['FLOW_future_sum'].values
X_test = df_test[flow_features_same].values
y_test = df_test['FLOW_future_sum'].values


X_train_seq, y_train_seq = create_sequences_sum(X_train, y_train, seq_length=15, horizon=15)
X_test_seq, y_test_seq = create_sequences_sum(X_test, y_test, seq_length=15, horizon=15)


X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train_seq.reshape(-1, X_train_seq.shape[2])).reshape(X_train_seq.shape)
X_test_scaled  = X_scaler.transform(X_test_seq.reshape(-1, X_test_seq.shape[2])).reshape(X_test_seq.shape)

y_train_scaled = y_scaler.fit_transform(y_train_seq.reshape(-1, 1))
y_test_scaled  = y_scaler.transform(y_test_seq.reshape(-1, 1))


n_features = X_train_scaled.shape[2]

checkpoint = ModelCheckpoint("model/LSTM_model_flow_same_best.keras", monitor='val_rmse', save_best_only=True, mode='min')

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(15, n_features), return_sequences=True),
    LSTM(64, activation='tanh', return_sequences=True),
    LSTM(64, activation='tanh'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
#model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1,verbose=1)
hist = model.fit(
                X_train_scaled, y_train_scaled,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr, checkpoint],
                verbose=1
            )

y_pred = model.predict(X_test_scaled)

rmse = mean_squared_error(y_test_scaled, y_pred, squared=False)
mae = mean_absolute_error(y_test_scaled, y_pred)
r2 = r2_score(y_test_scaled, y_pred)

print(f"LSTM 1 output (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

Epoch 1/100


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1460/1460 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - loss: 0.2134 - rmse: 0.4619 - val_loss: 0.2563 - val_rmse: 0.5063 - learning_rate: 0.0010
Epoch 2/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 23s 16ms/step - loss: 0.1929 - rmse: 0.4392 - val_loss: 0.2326 - val_rmse: 0.4823 - learning_rate: 0.0010
Epoch 3/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 23s 16ms/step - loss: 0.1865 - rmse: 0.4318 - val_loss: 0.2620 - val_rmse: 0.5119 - learning_rate: 0.0010
Epoch 4/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 22s 15ms/step - loss: 0.1817 - rmse: 0.4263 - val_loss: 0.2755 - val_rmse: 0.5249 - learning_rate: 5.0000e-04
Epoch 5/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - loss: 0.1803 - rmse: 0.4246 - val_loss: 0.2622 - val_rmse: 0.5120 - learning_rate: 5.0000e-04
456/456 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step
LSTM 1 output (FLOW) -> RMSE: 0.541, MAE: 0.328, R2: 0.772


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [244]:
split = int(len(X) * 0.8)
df_train, df_test = df_all[:split], df_all[split:]


X_train = df_train[speed_features_same].values
y_train = df_train['SPEED_future_mean'].values
X_test = df_test[speed_features_same].values
y_test = df_test['SPEED_future_mean'].values


X_train_seq, y_train_seq = create_sequences_sum(X_train, y_train, seq_length=15, horizon=15)
X_test_seq, y_test_seq = create_sequences_sum(X_test, y_test, seq_length=15, horizon=15)


X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train_seq.reshape(-1, X_train_seq.shape[2])).reshape(X_train_seq.shape)
X_test_scaled  = X_scaler.transform(X_test_seq.reshape(-1, X_test_seq.shape[2])).reshape(X_test_seq.shape)

y_train_scaled = y_scaler.fit_transform(y_train_seq.reshape(-1, 1))
y_test_scaled  = y_scaler.transform(y_test_seq.reshape(-1, 1))


n_features = X_train_scaled.shape[2]

checkpoint = ModelCheckpoint("model/LSTM_model_speed_same_best.keras", monitor='val_rmse', save_best_only=True, mode='min')

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(15, n_features), return_sequences=True),
    LSTM(64, activation='tanh', return_sequences=True),
    LSTM(64, activation='tanh'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
#model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1,verbose=1)
hist = model.fit(
                X_train_scaled, y_train_scaled,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr, checkpoint],
                verbose=1
            )

y_pred_scaled = model.predict(X_test_scaled)

y_pred = y_scaler.inverse_transform(y_pred_scaled)
y_true = y_scaler.inverse_transform(y_test_scaled)

rmse = mean_squared_error(y_true, y_pred, squared=False)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"LSTM 1 output (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

Epoch 1/100


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1460/1460 ━━━━━━━━━━━━━━━━━━━━ 23s 14ms/step - loss: 0.4509 - rmse: 0.6715 - val_loss: 0.3382 - val_rmse: 0.5815 - learning_rate: 0.0010
Epoch 2/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 0.4096 - rmse: 0.6400 - val_loss: 0.3145 - val_rmse: 0.5608 - learning_rate: 0.0010
Epoch 3/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 0.4014 - rmse: 0.6336 - val_loss: 0.3002 - val_rmse: 0.5479 - learning_rate: 0.0010
Epoch 4/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - loss: 0.3900 - rmse: 0.6245 - val_loss: 0.2989 - val_rmse: 0.5467 - learning_rate: 0.0010
Epoch 5/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - loss: 0.3878 - rmse: 0.6228 - val_loss: 0.3048 - val_rmse: 0.5521 - learning_rate: 0.0010
Epoch 6/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 0.3868 - rmse: 0.6219 - val_loss: 0.3204 - val_rmse: 0.5661 - learning_rate: 0.0010
Epoch 7/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 0.3868 - rmse: 0.6220 - val_loss: 0.3243 - val_rmse: 

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [245]:
rmse = mean_squared_error(y_true, y_pred, squared=False)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"LSTM 1 output (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

LSTM 1 output (SPEED) -> RMSE: 14.835, MAE: 7.751, R2: 0.644


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [205]:
split = int(len(X) * 0.8)
df_train, df_test = df_all[:split], df_all[split:]


X_train = df_train[flow_features_neighbour].values
y_train = df_train['FLOW_future_sum'].values
X_test = df_test[flow_features_neighbour].values
y_test = df_test['FLOW_future_sum'].values


X_train_seq, y_train_seq = create_sequences_sum(X_train, y_train, seq_length=15, horizon=15)
X_test_seq, y_test_seq = create_sequences_sum(X_test, y_test, seq_length=15, horizon=15)


X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train_seq.reshape(-1, X_train_seq.shape[2])).reshape(X_train_seq.shape)
X_test_scaled  = X_scaler.transform(X_test_seq.reshape(-1, X_test_seq.shape[2])).reshape(X_test_seq.shape)

y_train_scaled = y_scaler.fit_transform(y_train_seq.reshape(-1, 1))
y_test_scaled  = y_scaler.transform(y_test_seq.reshape(-1, 1))


n_features = X_train_scaled.shape[2]

checkpoint = ModelCheckpoint("model/LSTM_model_flow_neighbour_best.keras", monitor='val_rmse', save_best_only=True, mode='min')

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(15, n_features), return_sequences=True),
    LSTM(64, activation='tanh', return_sequences=True),
    LSTM(64, activation='tanh'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
#model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1,verbose=1)
hist = model.fit(
                X_train_scaled, y_train_scaled,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr, checkpoint],
                verbose=1
            )

y_pred = model.predict(X_test_scaled)

rmse = mean_squared_error(y_test_scaled, y_pred, squared=False)
mae = mean_absolute_error(y_test_scaled, y_pred)
r2 = r2_score(y_test_scaled, y_pred)

print(f"LSTM 1 output (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

Epoch 1/100


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


320/320 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.1518 - rmse: 0.3896 - val_loss: 0.2617 - val_rmse: 0.5116 - learning_rate: 0.0010
Epoch 2/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.1194 - rmse: 0.3455 - val_loss: 0.2489 - val_rmse: 0.4989 - learning_rate: 0.0010
Epoch 3/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.1129 - rmse: 0.3361 - val_loss: 0.2693 - val_rmse: 0.5190 - learning_rate: 0.0010
Epoch 4/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.1097 - rmse: 0.3313 - val_loss: 0.2553 - val_rmse: 0.5053 - learning_rate: 0.0010
Epoch 5/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.1082 - rmse: 0.3290 - val_loss: 0.2957 - val_rmse: 0.5438 - learning_rate: 0.0010
Epoch 6/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.1018 - rmse: 0.3191 - val_loss: 0.2295 - val_rmse: 0.4790 - learning_rate: 5.0000e-04
Epoch 7/100
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0995 - rmse: 0.3154 - val_loss: 0.2269 - val_rmse: 0.4764 - learning

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [206]:
df_test_peak= df_test.set_index("Datetime").between_time("07:15", "08:30").reset_index()
X_test_peak = df_test_peak[flow_features_neighbour].values
y_test_peak = df_test_peak['FLOW_future_sum'].values
X_test_seq_peak, y_test_seq_peak = create_sequences_sum(X_test_peak, y_test_peak, seq_length=15, horizon=15)

X_test_peak_scaled  = X_scaler.transform(X_test_seq_peak.reshape(-1, X_test_seq_peak.shape[2])).reshape(X_test_seq_peak.shape)
y_test_peak_scaled  = y_scaler.transform(y_test_seq_peak.reshape(-1, 1))

y_pred = model.predict(X_test_peak_scaled)

rmse = mean_squared_error(y_test_peak_scaled, y_pred, squared=False)
mae = mean_absolute_error(y_test_peak_scaled, y_pred)
r2 = r2_score(y_test_peak_scaled, y_pred)

print(f"LSTM 1 output (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
LSTM 1 output (FLOW) -> RMSE: 0.390, MAE: 0.266, R2: 0.727


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [243]:
split = int(len(X) * 0.8)
df_train, df_test = df_all[:split], df_all[split:]


X_train = df_train[speed_features_neighbour].values
y_train = df_train['SPEED_future_mean'].values
X_test = df_test[speed_features_neighbour].values
y_test = df_test['SPEED_future_mean'].values


X_train_seq, y_train_seq = create_sequences_sum(X_train, y_train, seq_length=15, horizon=15)
X_test_seq, y_test_seq = create_sequences_sum(X_test, y_test, seq_length=15, horizon=15)


X_scaler = StandardScaler()
#y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train_seq.reshape(-1, X_train_seq.shape[2])).reshape(X_train_seq.shape)
X_test_scaled  = X_scaler.transform(X_test_seq.reshape(-1, X_test_seq.shape[2])).reshape(X_test_seq.shape)

#y_train_scaled = y_scaler.fit_transform(y_train_seq.reshape(-1, 1))
#y_test_scaled  = y_scaler.transform(y_test_seq.reshape(-1, 1))

y_train_scaled=y_train_seq
y_test_scaled=y_test_seq

n_features = X_train_scaled.shape[2]

checkpoint = ModelCheckpoint("model/LSTM_model_speed_neighbour_best.keras", monitor='val_rmse', save_best_only=True, mode='min')

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(15, n_features), return_sequences=True),
    LSTM(64, activation='tanh', return_sequences=True),
    LSTM(64, activation='tanh'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
#model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1,verbose=1)
hist = model.fit(
                X_train_scaled, y_train_scaled,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=[early_stop, reduce_lr, checkpoint],
                verbose=1
            )

y_pred_scaled = model.predict(X_test_scaled)

#y_pred = y_scaler.inverse_transform(y_pred_scaled)
#y_true = y_scaler.inverse_transform(y_test_scaled)
y_pred=y_pred_scaled
y_true=y_test_scaled

rmse = mean_squared_error(y_true, y_pred, squared=False)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"LSTM 1 output (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

Epoch 1/100


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1460/1460 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - loss: 53527.5977 - rmse: 231.3603 - val_loss: 34466.0508 - val_rmse: 185.6503 - learning_rate: 0.0010
Epoch 2/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 22960.2344 - rmse: 151.5264 - val_loss: 12780.8682 - val_rmse: 113.0525 - learning_rate: 0.0010
Epoch 3/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - loss: 7528.6187 - rmse: 86.7676 - val_loss: 3142.8145 - val_rmse: 56.0608 - learning_rate: 0.0010
Epoch 4/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 1713.0857 - rmse: 41.3894 - val_loss: 586.0494 - val_rmse: 24.2085 - learning_rate: 0.0010
Epoch 5/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 591.1666 - rmse: 24.3139 - val_loss: 396.7042 - val_rmse: 19.9174 - learning_rate: 0.0010
Epoch 6/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 534.4307 - rmse: 23.1178 - val_loss: 398.0320 - val_rmse: 19.9507 - learning_rate: 0.0010
Epoch 7/100
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 53

C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
